# SVM

In this task, we will once again work with the MNIST training set. Prepare a training set matrix `X_train` consisting of the first 500 vectorized training samples of digits 1 and 2 each, and a corresponding label vector `y_train`. Use 1 and -1 for the labels.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from imageio import imread

N=500
digit_prefix=['d1','d2']
X_train=np.zeros((784,2*N))
for i,dp in enumerate(digit_prefix):
    for j in range(N):
        X_train[:,i*N+j]=np.float64(imread('mnist/'+dp+'/'+dp+'_'+'%04d.png'%(j+1)).ravel())
        
y_train=np.floor(np.arange(2*N)/N)*2-1

a) Write the function `simplesvm` which expects a training data matrix `X_train`, a training label vector `y_train` and a test data matrix `X_train` as its input. As a result, it returns the estimated test label vector `y_test`. To this end, employ `solvedualsvm` from the last lab course. Note that (8.29) in the lecture notes is overdetermined. You can exploit this to get a more robust estimation of $b$. Test your implementation with another 800 images from the MNIST data set.

In [2]:
from cvxopt import matrix, solvers

def solvedualsvm(H,y):
    y=y.squeeze()
    n=len(y)
    G=-np.eye(n).astype('double')
    A=y.reshape(1,n).astype('double')
    h=np.zeros((n,)).astype('double')
    b=np.zeros((1,)).astype('double')
    P=H.astype('double')
    q=-np.ones((n,)).astype('double')
    lambda_star=solvers.qp(matrix(P),matrix(q),matrix(G),matrix(h),matrix(A),matrix(b))
    return np.squeeze(lambda_star['x'])

def simplesvm(X_train,y_train, X_test=None):
    y_train=y_train.squeeze()
    H=np.dot(X_train.T,X_train)*np.expand_dims(y_train,0)*np.expand_dims(y_train,1)
    lambda_star=solvedualsvm(H,y_train)
    w=np.dot(X_train,lambda_star*y_train)
    lambda_sqrd=lambda_star**2
    b=np.dot(lambda_sqrd,np.dot(X_train.T,w)-y_train)/np.sum(lambda_sqrd)
    ret=np.sign(np.dot(X_test.T,w)-b)
    if X_test is None:
        ret=[w,b]
    return ret

N_test=400
digit_prefix=['d1','d2']
X_test=np.zeros((784,2*N_test))
for i,dp in enumerate(digit_prefix):
    for j in range(N_test):
        X_test[:,i*N_test+j]=np.float64(imread('mnist/'+dp+'/'+dp+'_'+'%04d.png'%(j+N+1)).ravel())
y_test=np.floor(np.arange(2*N_test)/N_test)*2-1
        
print('Success rate:', np.sum((simplesvm(X_train, y_train, X_test)*y_test+1))/(4*N_test))

ModuleNotFoundError: No module named 'cvxopt'

b) Implement `kernelsvm(X_train,y_train,sigma,X_test)` which works the same way as `simplesvm` - but in the feature space described by the Gaussian kernel. Test your implementation with different values for `sigma`.

In [ ]:
def kappa(X, Y, sigma):
    D2=np.expand_dims(np.sum(X**2, axis=0),1)+np.expand_dims(np.sum(Y**2, axis=0),0)-2*np.dot(X.T,Y)
    return np.exp(-D2/(2*sigma**2))

def kernelsvm(X_train,y_train,sigma, X_test):
    K=kappa(X_train,X_train,sigma)
    Ktrts=kappa(X_train,X_test,sigma)
    H=K*np.expand_dims(y_train,0)*np.expand_dims(y_train,1)
    lambda_star=solvedualsvm(H,y_train)
    lambday=lambda_star*y_train
    lambda_sqrd=lambda_star**2
    b=np.dot(lambda_sqrd,np.dot(K,lambday)-y_train)/np.sum(lambda_sqrd)
    return np.sign(np.dot(lambday,Ktrts)-b)

print('Success rate:', np.sum((kernelsvm(X_train, y_train, 2500, X_test)*y_test+1))/(4*N_test))